In [0]:
pip install kafka-python

Python interpreter will be restarted.
Collecting kafka-python
 Using cached kafka_python-2.0.2-py2.py3-none-any.whl (246 kB)
Installing collected packages: kafka-python
Successfully installed kafka-python-2.0.2
Python interpreter will be restarted.

In [0]:
%scala
System.setProperty("ssl.ca.location","dbfs:/FileStore/tables/cloudkarafka_apr2021.ca");

res1: String = dbfs:/FileStore/tables/cloudkarafka_apr2021.ca

In [0]:
%scala
System.clearProperty("java.security.auth.login.config");
System.clearProperty("sasl.jaas.config");
System.setProperty("java.security.auth.login.config", "dbfs:/FileStore/tables/cloudkarafka_apr2021.jaas");
System.setProperty("sasl.jaas.config", "dbfs:/FileStore/tables/cloudkarafka_apr2021.jaas");
println("auth.lang:" + System.getProperty("java.security.auth.login.config"));
println("sasl.jaas:" + System.getProperty("sasl.jaas.config"));

spark.sparkContext.addFile("dbfs:/FileStore/tables/cloudkarafka_apr2021.jaas")

auth.lang:dbfs:/FileStore/tables/cloudkarafka_apr2021.jaas
sasl.jaas:dbfs:/FileStore/tables/cloudkarafka_apr2021.jaas

In [0]:
topic = "1agne3o4-input"

print("====================== CSV Read ======================")
filePath = "/FileStore/tables/customer_applications_big.csv"
read = spark.read.format('csv').options(header='true').options(inferSchema='true').load(filePath)
read.show()

from kafka import KafkaProducer

print("read-type:", type(read))

def publishToKafka2(rows):
  print("publishToKafka2:", type(rows), rows)
  producer = KafkaProducer(
    bootstrap_servers=['omnibus-01.srvs.cloudkafka.com:9094','omnibus-03.srvs.cloudkafka.com:9094','omnibus-02.srvs.cloudkafka.com:9094'],
    client_id= "Kafka-Producer",
    security_protocol="SASL_SSL",
    sasl_mechanism="SCRAM-SHA-256",
    sasl_plain_username="1agne3o4",
    sasl_plain_password="hlifEhEaV6Gk3Fbu9m_cWzlhTezWUof6")
  for row in rows:
    print("publishToKafka2::Producing.....row:", type(row), row)
    #producer.send(topic, {"message", str(row.asDict())} )
    producer.send(topic, key=b'message', value=bytes(str(row.asDict()), "utf-8"))
    print("publishToKafka2::Sent.....row:", type(row), row)
  producer.flush() #Flush after queuing up all the rows.
  print("publishToKafka2::Flushed")

read.foreachPartition(publishToKafka2)
print("Produced Successfully - ALL DONE")

====================== CSV Read ======================
+---+-------+------+------+
 id|cust_id|app_id|active|
+---+-------+------+------+
 1| 1| 1| 1|
 2| 1| 2| 1|
 2| 2| 1| 1|
 2| 2| 2| 1|
 3| 3| 1| 1|
 4| 3| 2| 0|
 1| 1| 1| 1|
 2| 1| 2| 1|
 2| 2| 1| 1|
 2| 2| 2| 1|
 3| 3| 1| 1|
 4| 3| 2| 0|
 1| 1| 1| 1|
 2| 1| 2| 1|
 2| 2| 1| 1|
 2| 2| 2| 1|
 3| 3| 1| 1|
 4| 3| 2| 0|
 1| 1| 1| 1|
 2| 1| 2| 1|
+---+-------+------+------+
only showing top 20 rows

read-type: <class 'pyspark.sql.dataframe.DataFrame'>
Produced Successfully - ALL DONE